In [ ]:
from datasets import load_metric, load_dataset, Audio
from transformers import Wav2Vec2ForCTC, AutoProcessor, TrainingArguments, Trainer
import torch

from tqdm import tqdm

import re

from finetuning_util_hausa import preprocess_texts, preprocess_text, create_vocab_dict, create_data_collator, compute_metrics, ASRDataset

import json

cache_dir_fleurs ="/data/users/kashrest/lrl-asr-experiments/data/fleurs"

In [ ]:
"""from huggingface_hub import notebook_login

notebook_login()"""

cache_dir="/data/users/kashrest/lrl-asr-experiments/data/fleurs"

data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir)

# Inference - MMS-1b-all

In [ ]:
data[0]

In [ ]:
model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)

In [ ]:
processor.tokenizer.get_vocab()["hau"]

In [ ]:
#batch = processor.pad(input_features, padding=self.padding, max_length=self.max_length, pad_to_multiple_of=self.pad_to_multiple_of, return_tensors="pt")
processor.tokenizer.set_target_lang("hau")
model.load_adapter("hau")
transcriptions = []
gold_transcriptions = []
for elem in tqdm(data):
    inputs = processor(elem["audio"]["array"], sampling_rate=16_000, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs).logits
    ids = torch.argmax(outputs, dim=-1)[0]
    transcriptions.append(processor.decode(ids))
    gold_transcriptions.append(preprocess_text(elem["raw_transcription"]))
    # 'wachambuzi wa soka wanamtaja mesi kama nyota hatari zaidi duniani'
    # => In English: "soccer analysts describe Messi as the most dangerous player in the world"
    
with open("facebook_mms-1b-all/zero-shot/fluers_customized_preprocessing_test_output.jsonl", "w") as f:
    for transcription in transcriptions:
        json.dump(transcription, f)
        f.write("\n")

In [ ]:
len(transcriptions), len(gold_transcriptions)

In [ ]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
wer = wer_metric.compute(predictions=transcriptions, references=gold_transcriptions)
cer = cer_metric.compute(predictions=transcriptions, references=gold_transcriptions)
wer, cer

In [ ]:
transcriptions[0]

In [ ]:
gold_transcriptions[0]